In [5]:
import math
import csv
import ipyvolume.pylab as p3
import numpy as np
#WGS84 
##Equatorial Radius = 6378137
#Inverse Flattening = 298.257223563

a = 6378137
inv_f = 298.257224
f = 1/inv_f
g = (1 - f) * (1 - f)

input_path = 'fuse_to_obj.csv'
output_path = 'testproj.txt'
wgs84_data = []
local_coords_data = []

lat_ref = 45.90360309
long_ref = 11.02804799
alt_ref = 227.5475

def wgs84_to_ecef(lat, long, alt):
    lat_cos = math.cos(lat * math.pi / 180)
    lat_sin = math.sin(lat * math.pi / 180)

    long_cos = math.cos(long * math.pi / 180)
    long_sin = math.sin(long * math.pi / 180)

    d = 1 / math.sqrt(math.pow(lat_cos, 2) + g * math.pow(lat_sin, 2))

    x = (a * d + alt) * lat_cos * long_cos
    y = (a * d + alt) * lat_cos * long_sin
    z = (a * d * g + alt) * lat_sin

    return x, y, z

def ecef_to_enu(x, y, z, lat_ref, long_ref, alt_ref):
    lat_cos_ref = math.cos(lat_ref * math.pi / 180)
    lat_sin_ref = math.sin(lat_ref * math.pi / 180)

    long_cos_ref = math.cos(long_ref * math.pi / 180)
    long_sin_ref = math.sin(long_ref * math.pi / 180)

    d_ref = 1 / math.sqrt(math.pow(lat_cos_ref, 2) + g * math.pow(lat_sin_ref, 2))

    x_ref = (a * d_ref + alt_ref) * lat_cos_ref * long_cos_ref
    y_ref = (a * d_ref + alt_ref) * lat_cos_ref * long_sin_ref
    z_ref = (a * d_ref * g + alt_ref) * lat_sin_ref

    east = (-(x - x_ref) * long_sin_ref) + ((y - y_ref) * long_cos_ref)
    north = (-long_cos_ref * lat_sin_ref * (x - x_ref)) - (lat_sin_ref * long_sin_ref * (y - y_ref)) + (lat_cos_ref * (z - z_ref))
    up = (lat_cos_ref * long_cos_ref * (x - x_ref)) + (lat_cos_ref * long_sin_ref * (y - y_ref)) + (lat_sin_ref * (z - z_ref))

    return east, north, up

def wgs84_to_enu(lat, long, alt, lat_ref, long_ref, alt_ref):
    x, y, z = wgs84_to_ecef(lat, long, alt)

    return ecef_to_enu(x, y, z, lat_ref, long_ref, alt_ref)

def read_file(input_path):
    file = open(input_path, newline = '')
    reader = csv.reader(file)

    header = next(reader)
    data = []

    for row in reader:
        lat = float(row[0])
        long = float(row[1])
        alt = float(row[2])
        inten = float(row[3])

        data.append([lat, long, alt, inten])
    file.close()

    return data

def write_file(output_path):
    f = open(output_path, 'w')

    for t in local_coords_data:
        line = ' '.join(str(x) for x in t)
        f.write(line + '\n')
    f.close()

def coordTransformation():
    for pt in wgs84_data:
        e, n, u = wgs84_to_enu(pt[0], pt[1], pt[2], lat_ref, long_ref, alt_ref)
        local_coords_data.append([e, n, u, pt[3]])
        
    write_file(output_path)
    return local_coords_data

wgs84_data = read_file(input_path)
newData = coordTransformation()
zGround=[]
zLanes=[]
zAbove=[]
zPole=[]
#pole thoughts
#pick a z value that is high enough it could only be a pole. find points around that height. Once you find one, all points within certain tolerance of that point are in pole
for n in newData:
    if(n[0]<=.525*n[1]+13):
        if(n[0]>=.525*n[1]-8):
            if (n[2]>-2.2):
                zAbove.append(n)
            else:
                zGround.append(n)
for p in zGround:
    #use intensity to seperate markings
    if p[3]>=15:
        zLanes.append(p)
fig = p3.figure(width=1000)
allpoints = p3.scatter(np.array([n[0] for n in newData]), np.array([n[1] for n in newData]), np.array([n[2] for n in newData]), color='lightgrey', size=.05)
for p in zAbove:
    if 2<=p[2]<=3:
        zAbove.remove(p)
        zPole.append(p)
        for g in zAbove:
            #if similar x and y value to our 'anchor point', then in pole line
            if (-.6<=p[0]-g[0]<=.6) and (-.6<=p[1]-g[1]<=.6):
                zPole.append(g)
                zAbove.remove(g)
ground = p3.scatter(np.array([n[0] for n in zGround]), np.array([n[1] for n in zGround]), np.array([n[2] for n in zGround]), color='yellow', size=.05)
lanes = p3.scatter(np.array([n[0] for n in zLanes]), np.array([n[1] for n in zLanes]), np.array([n[2] for n in zLanes]), color='purple', size=.05)
above = p3.scatter(np.array([n[0] for n in zAbove]), np.array([n[1] for n in zAbove]), np.array([n[2] for n in zAbove]), color='red', size=.1)
pole = p3.scatter(np.array([n[0] for n in zPole]), np.array([n[1] for n in zPole]), np.array([n[2] for n in zPole]), color='blue', size=.1)
p3.style.set_style_dark()
p3.style.box_off()
p3.style.axes_off()
p3.squarelim()
p3.show()

